In [1]:
import hmmlearn
import numpy as np

In [2]:
import pickle 

def save_obj(obj, name):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
syllables_dict = load_obj("syllables_dict")

In [88]:

print(syllables)
print(sorted(np.unique(sentences)))
print(len(syllables.keys()))

{"'gainst": {'index': 0, 'syllabs': [1]}, "'greeing": {'index': 1, 'end_syllab': 1, 'syllabs': [2]}, "'scaped": {'index': 2, 'syllabs': [1]}, "'tis": {'index': 3, 'syllabs': [1]}, "'twixt": {'index': 4, 'syllabs': [1]}, 'a': {'index': 5, 'syllabs': [1]}, 'a-doting': {'index': 6, 'end_syllab': 2, 'syllabs': [3]}, 'abhor': {'index': 7, 'syllabs': [2]}, 'abide': {'index': 8, 'syllabs': [2]}, 'able': {'index': 9, 'syllabs': [2]}, 'about': {'index': 10, 'syllabs': [2]}, 'above': {'index': 11, 'syllabs': [2]}, 'absence': {'index': 12, 'syllabs': [2]}, 'absent': {'index': 13, 'syllabs': [2]}, 'abundance': {'index': 14, 'syllabs': [3]}, 'abundant': {'index': 15, 'syllabs': [3]}, 'abuse': {'index': 16, 'syllabs': [2]}, 'abused': {'index': 17, 'syllabs': [2]}, 'abuses': {'index': 18, 'syllabs': [3]}, 'abysm': {'index': 19, 'syllabs': [2]}, 'accents': {'index': 20, 'syllabs': [2]}, 'acceptable': {'index': 21, 'syllabs': [4]}, 'acceptance': {'index': 22, 'syllabs': [3]}, 'accessary': {'index': 23,

In [68]:
A = load_obj("trans_mat_epoch0")
O = load_obj("emiss_mat_epoch0")

emission_size = O.shape[1]
latent_size = A.shape[0]
print(A.shape, O.shape)
# latent_size = 10
# emission_size = 5
# ["I", "want", "orange", "door", "really"]
# A = np.random.random([latent_size, latent_size])
# O = np.random.random([latent_size, emission_size])
# A = A/np.tile(np.expand_dims(np.sum(A, axis = 1), axis = 1), latent_size)
# O = O/np.tile(np.expand_dims(np.sum(O, axis = 1), axis = 1), emission_size)
# O = O/np.tile(np.expand_dims(np.sum(O, axis = 1), axis = 1), latent_size)

(16, 16) (16, 3205)


In [69]:
print(list(syllables_dict.keys())[2])
print(syllables_dict[list(syllables_dict.keys())[2]])

'scaped
{'index': 2, 'syllabs': [1]}


In [85]:
# GENERATE SIMPLE SONNET NO RHYME SCHEME 
import random

sonnet = ""
all_emissions = list(range(emission_size))
all_states = list(range(latent_size))
emission_words = list(syllables_dict.keys())
max_syllab = 10
poem = []
poem_syllabs = []
for i in range(12):
    syllable_count = 0
    t = 0
    states = [0]*(max_syllab+10)
    lines = []
    line_syllabs = []
    while syllable_count < max_syllab:
        if syllable_count == 0:
            states[t] = random.choices(all_states, weights = [1/latent_size]*latent_size)
            idx = random.choices(all_emissions, weights = O[states[t]][0])[0]
            word = emission_words[idx]
            
        else:
            states[t] = random.choices(all_states, weights = A[states[t-1]][0])
            idx = random.choices(all_emissions, weights = O[states[t-1]][0])[0]
            word = emission_words[idx]
#             print(states, idx, word)
        
        # update syllable count
        word_syllables = syllables_dict[word]["syllabs"]
        syllab = random.choices(word_syllables, weights = [1/len(word_syllables)]*len(word_syllables))[0]
         
        # if exceeds syllable count, try again
        if (syllable_count + syllab) > max_syllab:
            while (syllable_count + syllab) > max_syllab:
                states[t] = random.choices(all_states, weights = A[states[t-1]][0])
                idx = random.choices(all_emissions, weights = O[states[t-1]][0])[0]
                word = emission_words[idx]
                word_syllables = syllables_dict[word]["syllabs"]
                syllab = random.choices(word_syllables, weights = [1/len(word_syllables)]*len(word_syllables))[0]
        syllable_count += random.choices(word_syllables, weights = [1/len(word_syllables)]*len(word_syllables))[0]
        sonnet += str(word)+ " "
        lines.append(word)
        line_syllabs.append(syllab)
        t += 1
    poem.append(lines)
    poem_syllabs.append(line_syllabs)
    sonnet += "\n"
    
print(sonnet)
print(np.array(poem))
print([len(poem[i]) for i in range(len(poem))])
print([len(poem_syllabs[i]) for i in range(len(poem_syllabs))])
print([sum(poem_syllabs[i]) for i in range(len(poem_syllabs))])
print(np.array(poem_syllabs))

10
10
10
10
10
10
10
10
10
10
10
10
my slay up put done to vulgar truth of 
fortune wound that right from time's nay hast that 
heart and i thee in my were't fortune heart 
the thy masonry they with this i to 
yet thee the my one yet me boast thy since 
do thy the that you self still no wide thing 
him women's others is passion which as 
neglected can the with of and women's 
why promise darling my year thy no world 
love vainly gored that hand false eyes his this 
be heart i painting heart's great this hast to 
the this profound any can bosom's is 

[list(['my', 'slay', 'up', 'put', 'done', 'to', 'vulgar', 'truth', 'of'])
 list(['fortune', 'wound', 'that', 'right', 'from', "time's", 'nay', 'hast', 'that'])
 list(['heart', 'and', 'i', 'thee', 'in', 'my', "were't", 'fortune', 'heart'])
 list(['the', 'thy', 'masonry', 'they', 'with', 'this', 'i', 'to'])
 list(['yet', 'thee', 'the', 'my', 'one', 'yet', 'me', 'boast', 'thy', 'since'])
 list(['do', 'thy', 'the', 'that', 'you', 'self', 'stil

In [89]:
sentences_idx = load_obj("sentences_idx_arr_with_punc_all")
sentences = load_obj("sentences_arr_with_punc_all")
syllables = load_obj("syllables_dict_with_punc")

In [95]:
# GENERATE SONNET WITH RHYME SCHEME

# Select rhyming words (6 base + 6 pairs)

rhyme_groups = {}
num_lines = len(sentences_idx)
num_sonnets = int(num_lines/14)
print(num_lines/14)

order = [0, 1, 0, 1, 2, 3, 2, 3, 4, 5, 4, 5, 6, 6]
for s in range(num_sonnets):
    sonnet = sentences[s*12:(s+1)*12]
    end_words = [0]*12
    for i in range(12):
        end_words[i] = sonnet[i][-1]
#     words1 = sentences[sonn]

153.92857142857142
